# Universo de simulação Autogerido

 Simulação de táxi. Uma frota de táxis, programada para iniciar seus turnos em horários diferentes, pode se comportar em conjunto. É necessário criar muitos táxis individuais, soltá-los em uma cidade cibernética e fazer com que relatem suas atividades. 

In [1]:
import pandas as pd
import numpy as np 



#geradores para simular os turnos dos táxis

def taxi_id_number(num_taxis): 

    arr= np.arange(num_taxis)
    np.random.shuffle(arr)
    for i in range(num_taxis): 
        yield arr[i]



In [2]:
ids= taxi_id_number(10)
print(next(ids))
print(next(ids))
print(next(ids))

8
5
9



Fará a iteração até gerar dez números, quando sairá do loop for dentro do gerador e lançará uma exceção StopIteration. 

In [3]:

def shift_info():
    # simulação de série temporal
    start_times_and_freqs = [(0,8), (8,30), (16, 15)]
    
    indices = np.arrange(len(start_times_and_freqs))
    
    while True: 
         idx = np.random.choice(indices, p=[0.25, 0.5,0.25])
         start= start_times_and_freqs[idx]
         yield(start[0], start[0] +7.5, start[1])
         

horas distintas do dia têm um número médio de viagens diferentes. 

Geração mais complexa

In [4]:

def taxi_process(taxi_id_generator, shift_info_generator):
    taxi_id = next(taxi_id_generator)
    shift_start, shift_end, shift_mean_trips = next(shift_info_generator)
    
    actual_trips = round(np.random.normal(loc = shift_mean_trips, scale =2))
    
    average_trip_time = 6.5/ shift_mean_trips*60
    
    # Converte tempo médio de viagem em minutos.
    between_events_time = 1.0/(shift_mean_trips-1)*60
    
    #Esta é uma cidade eficiente na qual os táxis raramente ficam sem rodar. 
    
    time = shift_start
    
    yield TimePoint(taxi_id, 'start shift', time)
    
    deltaT = np.random.poisson(between_events_time)/60
    time += deltaT
    
    for i in range(actual_trips):
        yield TimePoint(taxi_id, 'pick up   ', time)  
        deltaT = np.random.poisson(average_trip_time)/60        
        time += deltaT
        yield TimePoint(taxi_id, 'drop off  ', time)
        deltaT = np.random.poisson(between_events_time)/60
        time += deltaT
     
    deltaT = np.random.poisson(between_events_time) /60
    
    time += deltaT
    yield TimePoint(taxi_id,'end shift  ', time)

In [5]:

from dataclasses import dataclass

#simplifica o código, versão python3.7 @dataclass
@dataclass

class TimePoint:
     taxi_id: int
     name:str
     time: float
     
     
     def __lt__(self, other):
          return self.time < other.time
    

Tipo abstrato de dados, conhecido como first-in-first-out(FIFO)- objeto s sejam gerados a partir da estrutura de dados na mesma ordem em que foram inseridos nela. 

In [6]:
#método dunder , métodos integrados ára toda a classe. Base em seu tempo.

#Double under = dunder

#__lt__ comparar os TimePoints e str(para exibir TimePoints)

In [27]:
import queue

class Simulator: 
     def __init__(self, num_taxis):
         self._time_points = queue.PriorityQueue()
         taxi_id_generator = taxi_id_number(num_taxis)
         shift_info_generator = shift_info()
         self._taxis = [taxi_process(taxi_id_generator, shift_info_generator) for i in range(num_taxis)]
         
         self._prepare_run()
         #print('entrou 1')

    
     def _prepare_run(self):
         for t in self._taxis:
            # print('entrou 2')
             while True :
                try:
                   e = next(t)
                   self._time_points.put(e)
                except:
                   break
                   
     def run(self):
         sim_time=0
         print('entrou 3')
         print('vazio')
         print(sim_time)
         print(self)
         while sim_time <24:
             if self._time_points.empty():
                print('vazio')
                break
             p = self._time_points.get()
             sim_time = p.time
             print('entrou')
             print(p)


             

In [28]:
#fila de prioridades
sim = Simulator(1000)

sim.run() 

entrou 3
vazio
0
vazio


In [34]:
#fila de prioridades
sim = Simulator(10)

sim.run() 

entrou 3
vazio
0
vazio


In [35]:
sim._taxis 

[<generator object taxi_process at 0x78ce1188d6d0>,
 <generator object taxi_process at 0x78ce1188d800>,
 <generator object taxi_process at 0x78ce1188d930>,
 <generator object taxi_process at 0x78ce1188da60>,
 <generator object taxi_process at 0x78ce1188db90>,
 <generator object taxi_process at 0x78ce1188dcc0>,
 <generator object taxi_process at 0x78ce1188ddf0>,
 <generator object taxi_process at 0x78ce1188df20>,
 <generator object taxi_process at 0x78ce1188e050>,
 <generator object taxi_process at 0x78ce1188e180>]

https://github.com/PracticalTimeSeriesAnalysis/BookRepo/blob/master/Ch04/Ising.ipynb

In [37]:
#sim._time_points.get()


MCMC 
Simulações Monte Carlo e Cadeias de Markov

Simulação Monte Carlo é encontrar meios perspicazes de aplicar números aleatórios a situações que deveriam, em teoria, ter solução exata, mas que na prática sã mais fáceis de resolver com meios probabilísticos. A cadeia de Markov é útil em uma simulação Monte Carlo geral, ainda mais na simulação de séries temporais. 

Uma simulação Monte Carlo o ajudará a identificar como é uma determinada distribuição ou série de termos, mas não como esses termos devem evoluir ao longo dAo tempo. E é aqui a cadeia de Markov entra em cena, calculando a probabilidade de transição entre estados e, quando fatoramos isso, damos "passos" em vez de simplesmente calcular uma integral global. Agora podemos ter uma simualção de série temporal em vez de somente o cálculo de uma integral. 

- processo markov => a probabilidade de uma transição para um estado no futuro depende somente do estado presente (não de informações anteriores).

- Condições físicas espeíficas de uma distribuição de Boltzmann para energia: Tij/Tji = e ^-b(Ej-Ei)

ISING é o modelo mecânico clássico estatístico de ímãs.